In [ ]:
import json, re
import requests
from urlextract import URLExtract
import sys, gzip

utid = 'mkelle37'
base = {'model': 'https://huggingface.co/', 'data': 'https://huggingface.co/datasets/', 'source': 'https://'}
post = '/raw/main/README.md'
postGH = 'blob/master/README.md'

DOIpattern = r'\b(10\.\d{4,9}\/[-._;()/:A-Z0-9]+)\b/i'
extU = URLExtract()

def extractURLs(content):
    return extU.find_urls(content)

def extractDOIs(content):
    return re.findall(DOIpattern, content)

def run(tp):
    post0 = post if tp != 'source' else postGH
    with gzip.open(f"output/{utid}.json.gz", 'at', encoding='utf-8') as fo:
        with open(f"input/{utid}_{tp}", 'r') as f:
            for line in f:
                line = line.strip()
                id = line.split(';')[1] if tp == 'source' else line
                url = f"{base[tp]}{id}{post0}"
                r = requests.get(url)
                if r.status_code != 200:
                    continue
                
                content = r.text.replace('\n', ' ')
                urls = extractURLs(content)
                dois = extractDOIs(content)
                res = { 'ID': line, 'type': tp, 'url': url, 'content': content, 'links': urls, 'dois': dois }
                json.dump(res, fo)
                fo.write('\n')

run('model')
run('data')
run('source')